Listing the primes is not a very efficient way of computing $\pi(x)$ either in terms of time or space. One alternative method is Legendre's formula which counts the primes by the inclusion-exclusion principle.
\begin{equation}
    \pi(x) + 1 = \pi(\sqrt{x}) + [x] - \sum_{p_i \leq \sqrt{x}} \left[ \frac{x}{p_i} \right] + \sum_{p_i < p_j \leq \sqrt{x}} \left[ \frac{x}{p_i p_j} \right] - \sum_{p_i < p_j < p_k \leq \sqrt{x}} \left[ \frac{x}{p_i p_j p_k} \right] + \dots,
\end{equation}
where $[x]$ denotes the integer part of $x$ and $(p_i)_{i=1}^{\infty}$ is the sequence of all prime numbers.

This formula is not suited to practical computation because the number of terms involved increases rapidly with $x$ and because it requires knowledge of the list of primes up to $\sqrt{x}$. The multiple sums on the right-hand side of Legendre's formula can be interpreted as counting the number of integers $\leq x$ that are not divisible by any of the primes $\leq \sqrt{x}$.

Define $\psi(x, a)$ to be the numbers of integers $\leq x$ not divisible by any of the first $a$ primes. Then
\begin{equation}
    \psi(x, a) = [x] - \sum_{i \leq a} \left[ \frac{x}{p_i} \right] + \sum_{i < j \leq a} \left[ \frac{x}{p_i p_j} \right] - \sum_{i < j < k \leq a} \left[ \frac{x}{p_i p_j p_k} \right] + \dots,
\end{equation}
so Legendre's formula may be written as
\begin{equation}
    \pi(x) + 1 = \pi(\sqrt{x}) + \psi(x, \pi(\sqrt{x})).
\end{equation}
We alo have a recursion relation
\begin{equation}
    \psi(x, a) = \psi(x, a − 1) − \psi(x/p_a, a − 1)
\end{equation}
and $\psi(x, 0) = [x]$. This relation allows us to overcome the difficulty in programming the multiple summations.


In [58]:
import math
MAX_SQRT_X = 1000

def sieve(n):
    '''
    Generates a list of prime numbers up to n using the Sieve of Eratosthenes.
    '''
    primes_bool = [True] * (n + 1)
    if n >= 0:
        primes_bool[0] = False
    if n >= 1:
        primes_bool[1] = False
    for i in range(2, int(math.sqrt(n)) + 1):
        if primes_bool[i]:
            for multiple in range(i*i, n + 1, i):
                primes_bool[multiple] = False

    prime_numbers = []
    for i in range(2, n + 1):
        if primes_bool[i]:
            prime_numbers.append(i)
    return prime_numbers

PRIMES_CACHE = sieve(MAX_SQRT_X)

In [59]:
def psi(x, a, primes, memo, trace=None):
    '''
    Computes ψ(x, a) using the recursion relation with memoisation.
    '''
    # Use integer part of x for memoisation key.
    x_int = int(x)
    if (x_int, a) in memo:
        return memo[(x_int, a)]
    # Base case.
    if a == 0:
        return x_int
    # Recursion relation, the a-th prime pₐ is at index a-1 in the list.
    result = psi(x, a - 1, primes, memo, trace) - \
             psi(x / primes[a - 1], a - 1, primes, memo, trace)
    # Store the result in the memoisation cache.
    memo[(x_int, a)] = result
    # If tracing is enabled, log this computation.
    if trace is not None and (x_int, a) not in trace:
        trace[(x_int, a)] = result

    return result

def pi_legendre(x, trace=None):
    '''
    Computes the prime-counting function using the Legendre-psi formula.
    '''
    if x < 2:
        return 0

    sqrt_x = int(math.sqrt(x))
    # Find primes up to sqrt_x from our pre-computed list.
    primes_le_sqrt_x = []
    for p in PRIMES_CACHE:
        if p <= sqrt_x:
            primes_le_sqrt_x.append(p)
        else:
            break
    # a = π(√x) is the number of primes less than or equal to sqrt_x.
    a = len(primes_le_sqrt_x)
    # Initialize a memoization cache for this specific run.
    memo = {}
    # Calculate ψ(x, a) using the recursive function.
    psi_val = psi(x, a, primes_le_sqrt_x, memo, trace)
    # Apply the final formula.
    return a + psi_val - 1

steps = {}
print(pi_legendre(100, steps))
print(steps)

25
{(100, 1): 50, (33, 1): 17, (100, 2): 33, (20, 1): 10, (6, 1): 3, (20, 2): 7, (100, 3): 26, (14, 1): 7, (4, 1): 2, (14, 2): 5, (2, 1): 1, (0, 1): 0, (2, 2): 1, (14, 3): 4, (100, 4): 22}


This algorithm is still inefficient because we compute $\psi(x, a)$ many times over for small values of $a$. If we write
\begin{equation}
    m_k = \prod_{i=1}^k p_i,
\end{equation}
then we see that the pattern of multiples of the first $k$ primes repeats in a cycle of length $m_k$. Indeed,
\begin{equation}
    \psi(sm_k + t, k) = s\phi(m_k) + \psi(t, k),
\end{equation}
where $\phi(m_k)$ denotes the usual Euler $\phi$-function. If we pick a suitable value for $k$ and store the values $\psi(t, k)$ for $1 \leq t \leq m_k$, then we can curtail the recursion formula for $\psi(x, a)$ when $a$ is reduced to $k$ rather than $0$.


In [60]:
K = 6  # Our chosen value for k
FIRST_K_PRIMES = PRIMES_CACHE[:K]
M_K = 1
for p in FIRST_K_PRIMES:
    M_K *= p

# Calculate Euler's totient function φ(m_k)
PHI_M_K = M_K
for p in FIRST_K_PRIMES:
    PHI_M_K = PHI_M_K // p * (p - 1) # φ(30030) = 5760

# Pre-compute and store the values of ψ(t, K) for t up to M_K
# We use a simple sieve-like method for this one-time computation.
psi_k_table = list(range(M_K + 1))
for i in range(K):
    p = FIRST_K_PRIMES[i]
    for j in range(M_K, 0, -1):
        psi_k_table[j] -= psi_k_table[j // p]

memo_psi = {} # Memoisation cache for the main psi function

def psi(x, a):
    '''
    Computes ψ(x, a) using the recursion relation, memoisation,
    and the pre-computed table for ψ(t, K) as a new base case.
    '''
    x_int = int(x)
    if (x_int, a) in memo_psi:
        return memo_psi[(x_int, a)]
    # New base case: when a is reduced to K.
    if a == K:
        s = x_int // M_K
        t = x_int % M_K
        return s * PHI_M_K + psi_k_table[t]
    # Original base case: remains for a < K.
    if a == 0:
        return x_int
    # Recursive step for a > K.
    p_a = PRIMES_CACHE[a - 1]
    result = psi(x, a - 1) - psi(x / p_a, a - 1)
    memo_psi[(x_int, a)] = result

    return result

def pi_legendre_optimized(x):
    '''
    Computes π(x) using the optimized Legendre-psi formula.
    '''
    if x < 2:
        return 0
    sqrt_x = int(math.sqrt(x))
    a = 0
    for p in PRIMES_CACHE:
        if p <= sqrt_x:
            a += 1
        else:
            break
    global memo_psi
    memo_psi = {} # Clear memoization cache for each new π(x) calculation
    psi_val = psi(x, a)
    return a + psi_val - 1

for k in range(1, 6):
    print(k, [pi_legendre_optimized(n) for n in range(10**k, 10**(k+1) + 1, 10**k)])

1 [4, 8, 10, 12, 15, 17, 19, 22, 24, 25]
2 [25, 46, 62, 78, 95, 109, 125, 139, 154, 168]
3 [168, 303, 430, 550, 669, 783, 900, 1007, 1117, 1229]
4 [1229, 2262, 3245, 4203, 5133, 6057, 6935, 7837, 8713, 9592]
5 [9592, 17984, 25997, 33860, 41538, 49098, 56543, 63951, 71274, 78498]
